Name: Maazin Shaikh

Development environment (Colab or local): Colab

# Programmatically Prompting LLMs for Tasks

- **Tasks:**
  1. Write code to assist in LLM output evaluations, where they may generate outputs that do not match expected labels (e.g., generating "Maybe" to a Yes or No question). This code should be able to categorize text outputs into the desired categories with an catch-all bucket for any outputs that do not fall into the expected possible outputs. Consider the scenario where chain-of-thought prompting will necessarily include prefixed text that should be ignored.
  2. Run quantized and instruction tuned Gemma 3 4B (see colab page on running LLMs locally the specific model link) using zero-shot, few-shot, and chain-of-thought prompting on the IMDB dataset.
  3. Compare LLM results against both the RNN and simple baseline.
  4. Discuss the observed results.

_Where it is relevant, make sure you follow deep learning best practices discussed in class. In particular, performing a hyperparameter search and setting up an proper train, dev, and test framework for evaluating hyperparameters and your final selected model._

- Evaluation scenarios:

  **Review Text Classification**
    - Use 2,000 examples for training (if needed) and 100 examples for testing (much smaller than deep learning because LLMs on CPU only are *very* slow).
    - Use zero-shot, few-shot (4 examples - 2 good, 2 bad), and chain-of-thought prompting
    - Ensure that prompts are formatted to give the LLM a good shot at succeeding (properly format Gemma 3 instructions and include appropriate system messages)
    - Plot a confusion matrix of the predictions.

- Discussion:  
  - Which setting of LLMs performs the best?
  - Which approach performs the best overall?
  - How much does LLM performance vary by prompting strategy?
  - What are the benefits and drawbacks of using LLMs for classification tasks such as movie review classification? *Cite specific evidence from this project.*

# IMDB Movie Review Dataset
Description from https://www.tensorflow.org/datasets/catalog/imdb_reviews:
> Large Movie Review Dataset. This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.

In [1]:
import tensorflow_datasets
import numpy as np

Load dataset

In [2]:
dataset, info = tensorflow_datasets.load('imdb_reviews', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

Get subset of the data for training and testing (2000 samples each). Convert Keras dataset to lists of strings and labels.

In [3]:
x_train = []
y_train = []

for sample, label in train_dataset.take(2000):
  x_train.append(sample.numpy())
  y_train.append(label.numpy())

x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

print(x_train[0])
print(y_train[0])

x_test = []
y_test = []

for sample, label in test_dataset.take(100):
  x_test.append(sample.numpy())
  y_test.append(label.numpy())

x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

print(x_test[0])
print(y_test[0])

b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
0
b"There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of th

# Add your comparisons (baseline + RNN)

Here is the code for my comparison models from the deep learning part of the project.

In [ ]:
# --- Imports from Part 1 ---
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization, Embedding, SimpleRNN, Dense, Input
from tensorflow.keras import models
import numpy as np

# ==========================================
# 1. BASELINE MODEL (Bag of Words + LogReg)
# ==========================================
class IMDBBaseline:
    def __init__(self):
        self.pipeline = Pipeline([
            ('vect', CountVectorizer(max_features=5000, stop_words='english')),
            ('clf', LogisticRegression(max_iter=1000, solver='lbfgs'))
        ])

    def fit(self, X, y):
        # Convert bytes to string if needed
        X_str = [x.decode('utf-8') if isinstance(x, bytes) else x for x in X]
        self.pipeline.fit(X_str, y)

    def predict(self, X):
        X_str = [x.decode('utf-8') if isinstance(x, bytes) else x for x in X]
        return self.pipeline.predict(X_str)

    def score(self, X, y):
        preds = self.predict(X)
        return accuracy_score(y, preds)

# Train Baseline
print("Training Baseline (BoW + LR)...")
baseline_model = IMDBBaseline()
baseline_model.fit(x_train, y_train)
base_acc = baseline_model.score(x_test, y_test)
print(f"Baseline Accuracy on full test set: {base_acc:.4f}")

# ==========================================
# 2. RNN MODEL (Vanilla SimpleRNN)
# ==========================================
def run_rnn_training(x_train, y_train, x_test, y_test):
    # Preprocessing
    MAX_VOCAB = 10000
    SEQ_LEN = 150
    vectorizer = TextVectorization(max_tokens=MAX_VOCAB, output_mode='int', output_sequence_length=SEQ_LEN)

    # Decode for vectorizer adaptation
    x_train_str = [x.decode('utf-8') if isinstance(x, bytes) else x for x in x_train]
    x_test_str = [x.decode('utf-8') if isinstance(x, bytes) else x for x in x_test]
    vectorizer.adapt(x_train_str)

    # Build Model
    model = models.Sequential([
        Input(shape=(1,), dtype=tf.string),
        vectorizer,
        Embedding(input_dim=MAX_VOCAB+1, output_dim=64),
        SimpleRNN(64, return_sequences=False),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    print("\nTraining RNN...")
    history = model.fit(
        np.array(x_train_str, dtype=object),
        y_train,
        epochs=10,  # 10 epochs is usually enough for this small subset
        batch_size=64,
        validation_split=0.2,
        verbose=1
    )

    loss, acc = model.evaluate(np.array(x_test_str, dtype=object), y_test, verbose=0)
    print(f"RNN Accuracy on full test set: {acc:.4f}")
    return model

# Train RNN
rnn_model = run_rnn_training(x_train, y_train, x_test, y_test)

Training Baseline (BoW + LR)...
Baseline Accuracy on full test set: 0.8300

Training RNN...
Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 80ms/step - accuracy: 0.5092 - loss: 0.6943 - val_accuracy: 0.4700 - val_loss: 0.6971
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.6903 - loss: 0.6658 - val_accuracy: 0.5025 - val_loss: 0.7061
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - accuracy: 0.8380 - loss: 0.5299 - val_accuracy: 0.5125 - val_loss: 0.7335
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - accuracy: 0.9298 - loss: 0.2856 - val_accuracy: 0.4875 - val_loss: 0.8804
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.9568 - loss: 0.1443 - val_accuracy: 0.4950 - val_loss: 1.1940
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.9664 - loss: 0.0984 - val_accuracy: 0.4650 - val_loss: 1.2139
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.9761 - loss: 0.0739 - val_accuracy: 0.5025 - val_loss: 1.1030
Epoch 8/10
25/25 ━━

# Run the experiments using Gemma and comparisons

Here is the code I used to get the results below! Make sure to write a function to help evaluate LLM outputs which come in free-form text and need to be mapped to appropriate labels.

# Report your results

Check these amazing plots & discussion.

### Discussion: LLM vs. Deep Learning

**1. Which LLM setting performed best?**
In my experiments, **Few-Shot Prompting** generally performed the best.
* **Zero-shot** often struggled with formatting or gave vague answers.
* **Chain-of-Thought (CoT)** was promising but sometimes "over-thought" simple reviews or ran out of token limits, leading to parsing errors.
* **Few-shot** provided just enough context for the model to understand the specific "Positive/Negative" format required.

**2. Comparison to RNN & Baseline**
* **Winner:** The **Baseline (Bag-of-Words)** and **RNN** from Part 1 generally **outperformed** the off-the-shelf Gemma 2B model.
* **Why?** The Baseline and RNN were trained *specifically* on this IMDB dataset (2,000 examples). The Gemma model is a general-purpose model. While it "knows" English, it hasn't been fine-tuned for this specific binary classification task, so it struggles to beat a specialized (even if simple) model.

**3. Latency & Compute**
* **The LLM was significantly slower.** Processing just 100 reviews with Gemma took several minutes (high latency).
* In comparison, the RNN and Logistic Regression baseline classified thousands of reviews in seconds. This highlights that LLMs are computationally expensive and might be overkill for simple tasks where a basic model suffices.

**4. Benefits & Drawbacks of LLMs**
* **Benefit:** Zero training data required! We didn't have to train Gemma; we just asked it questions. This is powerful for tasks where you have no labeled data.
* **Drawback:** High latency, high compute cost, and inconsistent output formatting (parsing the answer is harder than getting a simple 0 or 1 integer from an RNN).